In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## from_template

In [6]:
from langchain.prompts import PromptTemplate

# from_template
template = "{word}로 삼행시 지어줘"
prompt_template = PromptTemplate.from_template(template)

# 생성
prompt = prompt_template.format(word="오이지")
prompt


'오이지로 삼행시 지어줘'

## 객체생성

In [7]:
# 객체생성
prompt_template = PromptTemplate(
    template=template,
    input_variables=["word"],
)

# 생성
prompt = prompt_template.format(word="오이지")
prompt

'오이지로 삼행시 지어줘'

## partial_variables + RunnablePassthrough

In [8]:
from datetime import datetime

def get_today():
    return datetime.now().strftime("%B %d")

get_today()

'May 20'

In [9]:
# partial_variables 고정값 사용
prompt_template = PromptTemplate(
    template="오늘 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 알려주세요.",
    input_variables=["n"],
    partial_variables={"today": get_today}
)

In [10]:
prompt_template.format(n=3)

'오늘 날짜는 May 20 입니다. 오늘이 생일인 유명인 3명을 알려주세요.'

In [12]:
from langchain_core.runnables import RunnablePassthrough

runnable_template = {"n" : RunnablePassthrough()} | prompt_template
runnable_template.invoke(5)

StringPromptValue(text='오늘 날짜는 May 20 입니다. 오늘이 생일인 유명인 5명을 알려주세요.')

## FewShotPromptTemplate

In [13]:
# FewShotPromptTemplate : 몇개의 예시를 주고, 그 예시를 바탕으로 문장을 생성하는 템플릿

from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {question}",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)


Question: 스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인


Question: 네이버의 창립자는 언제 태어났나요?

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question: 율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군


Question: 올드보이와 기생충의 감독이 같은 나라 출신인가요?

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예


Question: Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?


In [14]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0, 
    max_tokens=2048,  
    model_name="gpt-4o", 
)

print(llm.predict(final_prompt))

이 질문에 추가 질문이 필요합니다.

추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.

이제 계산을 통해 최종 답변을 도출할 수 있습니다:
1998년 - 1955년 = 43년

최종 답변은: Google이 창립된 연도인 1998년에 Bill Gates의 나이는 43세였습니다.


## Langchain Hub 에서 template 받아오기

In [15]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

In [16]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])